In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch import nn
import nltk
from nltk.tokenize import word_tokenize
import string
import re
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\фвьшт\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

из названия товара выделить бренд и тип

In [2]:
df = pd.read_csv("train_supervised_dataset.csv")
df.set_index("id", inplace=True)
df.head()

,name,good,brand
id,,,
0,Petmax Бантик леопард с красн розой 2шт,бантик,petmax
1,87191 Бусы для елки шарики_87191,бусы,NaN
2,Футболка Piazza Italia WR011446881,футболка,piazza italia
3,7) YI572-03X-ONE ЗАКОЛКА ДЛЯ ВОЛОС ДЛЯ ДЕВОЧКИ,заколка,NaN
4,Одежда (вес) 1500,одежда,NaN


In [3]:
len(df.good.unique())

2820

In [4]:
names = df["name"].apply(word_tokenize)

In [5]:
names.apply(len).argmax()

20779

In [6]:
names[names.apply(len).argmax()]

['136081',
 '005',
 'бежевый',
 '|',
 'мот',
 '|',
 'Пряжа',
 'Камтекс',
 '``',
 'Лотос',
 'травка',
 'стрейч',
 "''",
 '50гр',
 '.',
 '80м',
 '.',
 '(',
 '70',
 '%',
 'акрил',
 ',',
 '28',
 'п/а',
 ',',
 '2',
 '%',
 'лайкра',
 ')',
 '?',
 '00004',
 '?',
 '00000014021']

delete all digits and punctuations

In [25]:
punctuations = string.punctuation
punctuations

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [53]:
def eng_to_ru(word: str) -> str:
    if len(word) <= 1:
        return word
    a = ord('а')
    ru_alphabet = ''.join([chr(i) for i in range(a,a+33)])
    eng_alphabet = string.ascii_lowercase[:26]
    change = {
        "a": "а",
        "e": "е",
        "o": "о",
        "k": "к",
        "3": "з",
        "p": "р",
        "c": "с",
        "m": "м",
        "x": "х",
        "t": "т",
        "y": "у",
        "z": "з",
    }

    new_word = ""
    for i in range(len(word)):
        curr = word[i]
        if curr in eng_alphabet:
            if i - 1 < 0:
                prev_letter = "j"
            else:
                prev_letter = word[i - 1]
                
            if i + 1 >= len(word):
                next_letter = "j"
            else:    
                next_letter = word[i + 1]
            
            if next_letter in ru_alphabet or prev_letter in ru_alphabet:
                curr = change.get(curr, curr)
        new_word += curr
        
    return new_word
           
    
def only_ones(word: str) -> str:
    if word.isdigit() or re.match(r'^-?\d+(?:\.\d+)$', word):
        return "1"
    else:
        new_word = ""
        for i in range(len(word)):
            if word[i].isdigit():
                new_word += "1"
            else:
                new_word += word[i]
    return new_word


def only_words(lst: list) -> list:
    i = 0
    while i < len(lst):
        if pd.isna(lst[i]):
            lst[i] = "qklent"
        else:
            lst[i] = only_ones(lst[i])
            lst[i] = lst[i].lower()
            lst[i] = eng_to_ru(lst[i])
            
        x = lst[i]
        if len(x) < 2 or x.isdigit() or all(p in punctuations for p in x) or re.match(r'^-?\d+(?:\.\d+)$', x) is not None: #regex checks if x is decimal
            lst.pop(i)
        else:
            i += 1
    return lst
            

In [54]:
names = names.apply(only_words)
max_len = names.apply(len).max()
names[names.apply(len).argmax()]

['шприц',
 '1мл',
 '1-х',
 'комп',
 'sfm',
 'б/латекса',
 'стер',
 'игл',
 'g11',
 '1,11x11мм',
 'n11',
 'уп',
 'инд',
 'sfm',
 'hospital',
 'products',
 'gmbh',
 'германия']

In [56]:
brands = df.brand.fillna("qklent")
brands

id
0               petmax
1               qklent
2        piazza italia
3               qklent
4               qklent
             ...      
24995           sairme
24996        моя семья
24997           яшкино
24998            фаско
24999      крутой окер
Name: brand, Length: 25000, dtype: object

In [57]:
count = 0
for x in brands:
    if x == "qklent":
        count += 1
count #amount of nan brands

8505

In [58]:
brands = brands.apply(word_tokenize)
brands

id
0                [petmax]
1                [qklent]
2        [piazza, italia]
3                [qklent]
4                [qklent]
               ...       
24995            [sairme]
24996        [моя, семья]
24997            [яшкино]
24998             [фаско]
24999      [крутой, окер]
Name: brand, Length: 25000, dtype: object

In [59]:
df.good.isna().sum()

819

In [68]:
goods = df.good.fillna("qklent")
display(goods[goods == "товара нет"])
goods[goods == "товара нет"] = "qklent";
goods = goods.apply(word_tokenize)

id
12       товара нет
1322     товара нет
1353     товара нет
1431     товара нет
1939     товара нет
            ...    
24266    товара нет
24440    товара нет
24740    товара нет
24943    товара нет
24990    товара нет
Name: good, Length: 88, dtype: object

In [70]:
goods[24990]

['qklent']

In [71]:
df_new = pd.DataFrame(
    {
        "name": names,
        "good": goods,
        "brand": brands,
    }
)
df_new.head()

,name,good,brand
id,,,
0,"[petmax, бантик, леопард, красн, розой, 1шт]",[бантик],[petmax]
1,"[бусы, для, елки, шарики_11111]",[бусы],[qklent]
2,"[футболка, piazza, italia, wr111111111]",[футболка],"[piazza, italia]"
3,"[yi111-11x-one, заколка, для, волос, для, дево...",[заколка],[qklent]
4,"[одежда, вес]",[одежда],[qklent]


In [72]:
df_new.to_csv('tokenized_train.csv', index=False)

In [73]:
df_new[:10]

,name,good,brand
id,,,
0,"[petmax, бантик, леопард, красн, розой, 1шт]",[бантик],[petmax]
1,"[бусы, для, елки, шарики_11111]",[бусы],[qklent]
2,"[футболка, piazza, italia, wr111111111]",[футболка],"[piazza, italia]"
3,"[yi111-11x-one, заколка, для, волос, для, дево...",[заколка],[qklent]
4,"[одежда, вес]",[одежда],[qklent]
5,"[русалочка, губка, трио, трехслой.1, шт, люкс]",[губка],[русалочка]
6,"[11-111, обои/art/п.п, белые, на, флизе, 1,11,...",[обои],[qklent]
7,"[квас, очаковский, пряная, зима, 1л]",[квас],[очаковский]
8,"[нб, маска+п/cosy/м-ив]",[маска],[cosy]


In [73]:
df_new[:10]

,name,good,brand
id,,,
0,"[petmax, бантик, леопард, красн, розой, 1шт]",[бантик],[petmax]
1,"[бусы, для, елки, шарики_11111]",[бусы],[qklent]
2,"[футболка, piazza, italia, wr111111111]",[футболка],"[piazza, italia]"
3,"[yi111-11x-one, заколка, для, волос, для, дево...",[заколка],[qklent]
4,"[одежда, вес]",[одежда],[qklent]
5,"[русалочка, губка, трио, трехслой.1, шт, люкс]",[губка],[русалочка]
6,"[11-111, обои/art/п.п, белые, на, флизе, 1,11,...",[обои],[qklent]
7,"[квас, очаковский, пряная, зима, 1л]",[квас],[очаковский]
8,"[нб, маска+п/cosy/м-ив]",[маска],[cosy]
